In [1]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import random
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [2]:
print(torch.cuda.is_available())
torch.cuda.set_device(2)

True


In [3]:
# Define a convolutional block
# A single layer of convolution if shortcut == False, and...
# Two layers of convolution and a residual convolution otherwise.

class ConvBlock(nn.Module):
    
    # midChannels will be of no use if shortcut == False
    def __init__(self, inChannels, midChannels, outChannels, 
                 kernelSize, stride = 1, padding = 0, bias = True, shortcut = False):
        super(ConvBlock, self).__init__()
        if shortcut is False:
            self.left = nn.Sequential(
                nn.Conv2d(inChannels, outChannels, 
                          kernelSize, stride, padding, groups = inChannels, bias = bias),
                nn.BatchNorm2d(outChannels)
            )
            self.right = None
        else:
            self.left = nn.Sequential(
                nn.Conv2d(inChannels, midChannels, 
                          kernelSize, stride, padding, groups = inChannels, bias = bias),
                nn.BatchNorm2d(midChannels),
                nn.ReLU(inplace = True),
                nn.Conv2d(midChannels, outChannels, 
                          kernelSize, stride, padding, groups = midChannels, bias = bias),
                nn.BatchNorm2d(outChannels)
            )
            self.right = nn.Sequential(
                nn.Conv2d(inChannels, outChannels, 
                          kernelSize, stride, padding, groups = inChannels, bias = bias),
                nn.BatchNorm2d(outChannels)
            )
        
    def forward(self, input):
        out = self.left(input)
        if self.right is not None:
            out += self.right(input)
        return F.relu(out)
    

In [4]:
# Define a fully connected block
# A single layer if shortcut == False, and...
# Two layers and a residual layer otherwise.

class FCBlock(nn.Module):
    
    # midChannels will be of no use if shortcut == False
    def __init__(self, inNodes, midNodes, outNodes, shortcut = False):
        super(FCBlock, self).__init__()
        if shortcut is False:
            self.left = nn.Sequential(
                nn.Linear(inNodes, outNodes)
            )
            self.right = None
        else:
            self.left = nn.Sequential(
                nn.Linear(inNodes, midNodes),
                nn.ReLU(inplace = True),
                nn.Linear(midNodes, outNodes)
            )
            self.right = nn.Sequential(
                nn.Linear(inNodes, outNodes)
            )
            
    def forward(self, input):
        out = self.left(input)
        if self.right is not None:
            out += self.right(input)
        return F.relu(out)
        

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [ ]:
class ChunkSampler(sampler.Sampler):
    """Samples elements randomly from the given index list. 
    """
    def __init__(self, index_list):
        self.index = index_list
        self.length = len(index_list)

    def __iter__(self):
        return iter(self.index)

    def __len__(self):
        return self.length


In [ ]:
gpuDtype = torch.cuda.FloatTensor

trainList = []
valList = []
def checkAccuracy(model, trainLoader, valLoader):
    numCorrect = 0
    numSamples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in trainLoader:
        with torch.no_grad():
            xVar = Variable(x.type(gpuDtype))
            scores = model(xVar)
            _, preds = scores.data.cpu().max(1)
            numCorrect += (preds == y).sum()
            numSamples += preds.size(0)
    acc = float(numCorrect) / numSamples
    trainList.append(acc)
    print('Train: %d / %d correct (%.2f%%)' % (numCorrect, numSamples, 100 * acc))
    
    numCorrect = 0
    numSamples = 0
    for x, y in valLoader:
        with torch.no_grad():
            xVar = Variable(x.type(gpuDtype))
            scores = model(xVar)
            _, preds = scores.data.cpu().max(1)
            numCorrect += (preds == y).sum()
            numSamples += preds.size(0)
    acc = float(numCorrect) / numSamples
    valList.append(acc)
    print('Val: %d / %d correct (%.2f%%)' % (numCorrect, numSamples, 100 * acc))
    
def train(model, lossFunc, optimizer, numEpochs = 1, 
          lpReg = {}, lambdaFCB = 0.0, lambdaConvB = 0.0,
          printEvery = 100, checkEveryEpoch = True):
    for epoch in range(numEpochs):
        print('Starting epoch %d / %d' % (epoch + 1, numEpochs))
        model.train()
        for t, (x, y) in enumerate(trainLoader):
            xVar = Variable(x.type(gpuDtype))
            yVar = Variable(y.type(gpuDtype).long())
            scores = model(xVar)            
            loss = lossFunc(scores, yVar)
            
            weights = {}
            for name, param in model.named_parameters():
                if 'bias' not in name:
                    #print(name)
                    #print(param)
                    weights[name] = param 
                    for (p, lamb) in lpReg.items():
                        loss += lamb * (torch.norm(param, int(p)) ** int(p))
                        
            if abs(lambdaFCB) > 1e-9:
                FCB1WF1dotWS = weights['blocks.5.left.0.weight'].mm(weights['blocks.5.right.0.weight'].t())
                FCB2WF1dotWS = weights['blocks.6.left.0.weight'].mm(weights['blocks.6.right.0.weight'].t())
                loss += lambdaFCB * torch.norm(FCB1WF1dotWS, 2)
                loss += lambdaFCB * torch.norm(FCB2WF1dotWS, 2)
            
#             print('Before: %f' % loss)
            if abs(lambdaConvB) > 1e-9:
                Conv_11 = 0.0
                Conv_12 = 0.0
                Conv_21 = 0.0
                Conv_22 = 0.0

                for i in range(9):
                    for j in range(3):
                        tmp = weights['blocks.0.left.0.weight'][i]-weights['blocks.0.right.0.weight'][i*3+j]
                        Conv_11 += torch.norm(tmp,2)
                for i in range(27):
                    tmp = weights['blocks.0.left.3.weight'][i]-weights['blocks.0.right.0.weight'][i]
                    Conv_12 += torch.norm(tmp,2)

                for i in range(81):
                    for j in range(3):
                        tmp = weights['blocks.2.left.0.weight'][i]-weights['blocks.2.right.0.weight'][i*3+j]
                        Conv_22 += torch.norm(tmp,2)
                for i in range(243):
                    tmp = weights['blocks.2.left.3.weight'][j]-weights['blocks.2.right.0.weight'][i]
                    Conv_22 += torch.norm(tmp,2)

                loss += lambdaConvB * (Conv_11 + Conv_12 + Conv_21 / 9 + Conv_22 /9) 
#             print('After: %f' %  loss)
            if printEvery > 0 and (t + 1) % printEvery == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if checkEveryEpoch:
            checkAccuracy(model, trainLoader, valLoader)

In [ ]:
def showFigure(markLast = 0):
    fig = plt.figure(figsize = (15, 8))
    NUM_EPOCHS = len(trainList)
    pt, = plt.plot(range(1, NUM_EPOCHS + 1), trainList, c = 'red')
    pv, = plt.plot(range(1, NUM_EPOCHS + 1), valList, c = 'blue')
    for i in range(NUM_EPOCHS):
        plt.text(i + 1, trainList[i] + ((i % 2) - 0.8) * 0.005, '%.4f' % trainList[i], ha = 'center', va = 'bottom', fontsize = 9)
        plt.text(i + 1, valList[i] + ((i % 2) - 0.8) * 0.005, '%.4f' % valList[i], ha = 'center', va = 'bottom', fontsize = 9)
        if markLast > 0 and (i == NUM_EPOCHS - markLast or i == NUM_EPOCHS - 1):
            plt.axvline(x = i + 1, color = 'green', linewidth = 2)
        else:
            plt.axvline(x = i + 1, color = 'lightgrey', linewidth = 1, linestyle = '--')
    plt.xticks(range(1, NUM_EPOCHS + 1))
    plt.legend([pt, pv], ['Train', 'Val'], loc = 'upper left')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

def showDeltaAcc(numEpochs):
    deltaAcc = 0.0
    avgValAcc = 0.0
    totalEpochs = len(trainList)
    for i in range(totalEpochs - numEpochs, totalEpochs):
        deltaAcc += trainList[i] - valList[i]
        avgValAcc += valList[i]
    deltaAcc /= numEpochs
    avgValAcc /= numEpochs
    print('Delta accuracy of the last %d epochs is: %.4f' % (numEpochs, deltaAcc))
    print('Average validation accuracy of the last %d epochs is: %.4f' % (numEpochs, avgValAcc))

In [ ]:
class ResNet(nn.Module):
    
    def __init__(self):
        super(ResNet, self).__init__()
        
        self.blocks = nn.Sequential(
            ConvBlock(inChannels = 3, midChannels = 9, outChannels = 27, 
                      kernelSize = 3, padding = 1, shortcut = True),
            nn.MaxPool2d(kernel_size = 2),
            ConvBlock(inChannels = 27, midChannels = 81, outChannels = 243, 
                      kernelSize = 3, padding = 1, shortcut = True),
            nn.MaxPool2d(kernel_size = 2),
            
            Flatten(),
            FCBlock(inNodes = 15552, midNodes = 2592, outNodes = 432, 
                    shortcut = True),
            FCBlock(inNodes = 432, midNodes = 72, outNodes = 24, 
                    shortcut = True),
            FCBlock(inNodes = 24, midNodes = 0, outNodes = 10, 
                    shortcut = False)
            
        )
    
    def forward(self, input):
        out = self.blocks(input)
        return out

In [ ]:
DATASET_SIZE = 50000

numTrain = 20000
numVal = 1000

#random.seed(666)
index_all = random.sample(range(DATASET_SIZE), numTrain + numVal)
index_train = index_all[ : numTrain]
index_val = index_all[numTrain : ]

trainData = datasets.CIFAR10('./data', train = True,
                           transform = transforms.ToTensor())
trainLoader = DataLoader(trainData, batch_size = 64, 
                              sampler = ChunkSampler(index_train))
        
valData = datasets.CIFAR10('./data', train = True,
                           transform = transforms.ToTensor())
valLoader = DataLoader(valData, batch_size = 64, 
                            sampler = ChunkSampler(index_val))

testData = datasets.CIFAR10('./data', train = False,
                          transform = transforms.ToTensor())
testLoader = DataLoader(testData, batch_size = 64)

In [ ]:
#TORCH_SEED = 666
#torch.manual_seed(TORCH_SEED)
#torch.cuda.manual_seed(TORCH_SEED)

model = ResNet().type(gpuDtype)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

lpReg = {'2':1e-4}
train(model, criterion, optimizer, numEpochs = 50,
      lpReg = lpReg, lambdaFCB = 0.0, lambdaConvB = -1e-4,
      printEvery = 0, checkEveryEpoch = True)

showDeltaAcc(numEpochs = 10)
showFigure(markLast = 10)

Starting epoch 1 / 50
Train: 7105 / 20000 correct (35.52%)
Val: 337 / 1000 correct (33.70%)
Starting epoch 2 / 50
Train: 9434 / 20000 correct (47.17%)
Val: 429 / 1000 correct (42.90%)
Starting epoch 3 / 50
Train: 10258 / 20000 correct (51.29%)
Val: 459 / 1000 correct (45.90%)
Starting epoch 4 / 50
Train: 10218 / 20000 correct (51.09%)
Val: 457 / 1000 correct (45.70%)
Starting epoch 5 / 50
Train: 10741 / 20000 correct (53.71%)
Val: 464 / 1000 correct (46.40%)
Starting epoch 6 / 50
Train: 11046 / 20000 correct (55.23%)
Val: 460 / 1000 correct (46.00%)
Starting epoch 7 / 50
Train: 11236 / 20000 correct (56.18%)
Val: 456 / 1000 correct (45.60%)
Starting epoch 8 / 50
Train: 11229 / 20000 correct (56.15%)
Val: 450 / 1000 correct (45.00%)
Starting epoch 9 / 50
Train: 12418 / 20000 correct (62.09%)
Val: 474 / 1000 correct (47.40%)
Starting epoch 10 / 50
Train: 12561 / 20000 correct (62.80%)
Val: 477 / 1000 correct (47.70%)
Starting epoch 11 / 50
Train: 12469 / 20000 correct (62.34%)
Val: 467 /